In [1]:
import glob
import os
from tqdm import tqdm
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pickle
import numpy as np
from gensim.models import word2vec
import logging
import datetime
%matplotlib inline

ModuleNotFoundError: No module named 'gensim'

In [2]:
java_path = "./JavaDataSet/small/training/"
path = "./PCV_Method/"

In [3]:
# データをバッチ化するための関数
def train2batch(input_data, output_data, batch_size=100):
    input_batch = []
    output_batch = []
    input_shuffle, output_shuffle = shuffle(input_data, output_data)
    for i in range(0, len(input_data), batch_size):
        input_batch.append(input_shuffle[i:i+batch_size])
        output_batch.append(output_shuffle[i:i+batch_size])
    return input_batch, output_batch

In [4]:
# Encoderクラス
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, weight):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.word_embeddings.weight = nn.Parameter(torch.from_numpy(weight))
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

    def forward(self, sequence):
        embedding = self.word_embeddings(sequence)
        # Many to Oneなので、第２戻り値を使う
        _, state = self.lstm(embedding)
        # state = (h, c)
        return state

In [5]:
# Decoderクラス
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, weight):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
#         self.word_embeddings.weight = nn.Parameter(torch.from_numpy(weight))
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        # LSTMの128次元の隠れ層を13次元に変換する全結合層
        self.hidden2linear = nn.Linear(hidden_dim, vocab_size)

    def forward(self, sequence, encoder_state):
        embedding = self.word_embeddings(sequence)
        # Many to Manyなので、第１戻り値を使う。
        # 第２戻り値は推論時に次の文字を生成するときに使います。
        output, state = self.lstm(embedding, encoder_state)
        output = self.hidden2linear(output)
        return output, state

In [6]:
def code2index(code, vocab_dict):
    return [vocab_dict[node] if node in vocab_dict else 0 for node in code]

In [7]:
def sep_method_name(mn):
#     print(mn)
    m = []
    start_i = 0
    end_i = 0
    for i, c in enumerate(mn):
        if (c == '_' and start_i+1!=i):
            m.append(mn[start_i:i])
            start_i = i + 1
        elif (c.isupper() and start_i!=i):
            m.append(mn[start_i:i])
            start_i = i
    
    m.append(mn[start_i:])
    return m 

In [8]:
def method2index(method_name):
#     mn_list = sep_method_name(method_name)
    for mn2 in method_name:
        if mn2 in word2index: continue
        word2index[mn2] = len(word2index)
    return [word2index[mn2] if mn2 in word2index else 0 for mn2 in method_name]

run
Time
Define


[9, 10, 11]

In [9]:
with open(path + 'data/code_df_all.pickle', 'rb') as f:
    code_data_frame = pickle.load(f)

In [9]:
with open(path + 'pcv/vocab_dict.dict', 'rb') as file:
     pcv_vocab_dict = pickle.load(file)
with open(path + 'pcv/wv_model.model', 'rb') as file:
     pcv_model = pickle.load(file)
with open(path + 'pcv/weight.weight', 'rb') as file:
     pcv_weight = pickle.load(file)
with open(path + 'pcv/list.dict', 'rb') as file:
     pcv_list = pickle.load(file)

In [11]:
print(len(pcv_vocab_dict))
print(len(pcv_weight))
print(len(pcv_list))


623
623
1000


In [10]:
with open(path + 'pcv/train_x.pickle', 'rb') as file:
     pcv_train_x = pickle.load(file)
with open(path + 'pcv/train_y.pickle', 'rb') as file:
     pcv_train_y = pickle.load(file)
with open(path + 'pcv/test_x.pickle', 'rb') as file:
     pcv_test_x = pickle.load(file)
with open(path + 'pcv/test_y.pickle', 'rb') as file:
     pcv_test_y = pickle.load(file)
with open(path + 'pcv/word2index.dict', 'rb') as file:
     word2index = pickle.load(file)

In [29]:
len(pcv_vocab_dict)

623

In [14]:
code_data_frame = code_data_frame.sample(10000)

In [11]:
# 単語のベクトル次元数
EMBEDDING_DIM = len(pcv_weight[0])
# 隠れ層の次元数
HIDDEN_DIM = 128
# データ全体の単語数
VOCAB_SIZE = len(pcv_vocab_dict)
DEC_VOCAB_SIZE = len(word2index)

In [16]:
pcv_datasets = pd.DataFrame(columns=["code", "method_name"])
print('----------datasets loading----------')
for index, row in tqdm(code_data_frame.iterrows()):
    pcv = []
    for node in row['Parent Child Vector']:
        pcv.append(node[1])
    level = sep_method_name(row['method_name'])
    s = pd.Series([pcv, level], index=pcv_datasets.columns)
    pcv_datasets = pcv_datasets.append(s, ignore_index=True)

71it [00:00, 701.78it/s]

----------datasets loading----------
resetPortletInvocationWhitelist
getJdkType
run
getChildren
isFirst
getClassName
type
saveExtensions
getComposite
jvmStats
from
getDDMFormFieldTypeSettings
handleMessage
getAdvice
executor
setCategoryId
print
createDDMForm
setReviewDate
configurePointcutParameters
setName
getId
getBufferEnd
initComponent
onComplete
isOptionEnabled
clone
getMainXmlMapping
addCompanyScopePermissions
compute
run
getContactPoints
getPasswordEncryptor
testGetDescendants
testInterf
setReadOnly
testMostSpecificVarargsCase
visitSingleGroupingSet
getNextSiblingInTag
testPublicScheduleMethod
listenerAdded
setGroupService
getPosition
equals
isResolved
ignored
isStagingGroup
filterCountByG_P
paste
setOutParameterNames
populate
withFormatType
getFieldName
getModelClass
findByKeywords
setPattern
registerTransformers130
prepareLocalizedFieldsForImport
testEquatableAll
openDirectTCPIPChannel
getModuleToBeRenamed
testT125
getHighlightPolicy
getCalendar
process
buildVisitor
getClassPK

219it [00:00, 718.31it/s]

countAll
getUserId
bar
getNewIntValue
testAfterNew8
getConfigRoot
buildVisitor
findByUuid_C
applicationActivated
hashCode
valueType
getMethod
testSystemPropertyReplacement
isCopyVisible
foo
createPageSizesCombo
clearCache
testT26
removeByG_N
withNullValue
testDependsOnGroupDontIncludeForeignClass
getDefaultLanguageId
getAttachmentAsDataHandler
getElementPredicate
getTypes
testSurroundWithTryCatchProperties
_getFormatLabel
multiBind
fetchByUuid_C_Last
run
isValid
findByUuid_C
ensureContentOrdered
toBytes
setUserId
testMergingMappings
addPortlet
isValid
definitelyRunningAsRoot
rangeQuery
fun
getSqlType
setUserTeams
testReverse
getByG_C_PrevAndNext
getParserInput
createReplaceWithImportFix
search
dispose
clusterChanged
getImportDataGroupElement
genPasswordKey
invokeImpl
buildFixes
getIndexer
toUnescapedModel
writeReport
findByG_R
createRecordReader
getDescriptor
getTreeStep
modifiedService
removeByCompanyId
hashCode
assertExpectedTableLayoutHandle
close
getNoMoreSplits
foo
removeAttribute

410it [00:00, 819.35it/s]

...

update
encryptUserId
foo
setJbossHomeDir
clear
createDDMFormRendererHelper
getContainerModelId
setExpandoBridgeAttributes
getWorkedTools
commitAndRunReadAction
scrollCustomQuery
testSelfReference
deserialize
checkAutoPopup
testParseSimpleLinkTagWithoutDescription
validate
getParent
clear
findBySubtype_Last
toUnescapedModel
checkFilesAreInList
setModuleServiceLifecycle
getProjectDir
addLocationStrategy
customizeCellRenderer
setShoppingItemFieldPersistence
delete
getHashCode
run
getGroupThreads
assertGetRecordsOptional
getByCompanyId_PrevAndNext
getPersistence
getBookmarksEntries
customize
updateOffsetAndDuration
testNamedParameterUsingInvalidQuestionMarkPlaceHolders
fun
ensureFinished
getRuleGroupInstance
i
destroyExecutor
getColumnMetadata
processExportPortletPreferences
getDisplayName
findAll
getDLFileEntryType
createSelector
setLDAPExportConfigurationProvider
exec
createSocialActivityAchievement
getChildren
getOrderByComparator
testHistoryOfLukasz
testErrorEndsAtNewline
oneOf
getProp

9884it [00:12, 680.98it/s]

getFavoritesEditor
getMapKeyType
baz297
removeQueues
getFragments
isFinderCacheEnabled
remove
assertEmptyMappings
dynamicQuery
getModel
getResourceBlockId
mouseEntered
getPositionChangeX
getOrders
testExistingReadOnlyAfterSetSessionModifiableExistingEntityReadOnly
getAssetTagLocalService
writeStaticResourceAttribute
deleteGroupComments
getUserGroup
testCommonTermsQueryOnAllField
requireChild
findDeepestSuperMethods
getTargetType
isValid
testParseVersionsSet3
_generateImages
characters
method2529
setRestrictionType
setConfigurationFactory
drawString
setId
addGroupUserGroup
toString
getAssembler
getProjects
test
beforeCharDeleted
_tryTerminate
createHelpAction
performRefactoring
addCommandListener
findByUuid_C
getIndexableActionableDynamicQuery
getDescription
surroundStatement
assertCssValue
getPortalLocalService
createFloatingPointExpression
addSorter
findByClassNameId
getSignature
updateLastPublishDate
getFixedChildDescription
getActionableDynamicQuery
visitSemiJoin
resetOriginalValues

10000it [00:12, 798.29it/s]

toString
ignoreUnknownPathExtensions
createMutation
compute
processTemplate
getTaskID
getFiles
setValue
setStatusByUserId
addReservedEl
defineSchema
isModified
paintIcons
getDisplayName
createCalendarBooking
getTypeParameters
testDependenciesThroughPropertiesInInnerBean
deletePersistedModel
getControlPanelPortletURL
parseObjectFactoryBinding
setEditorProvider
testSPIRegistration
getLocalAddresses
setListable
prepareTestInstance
getSubprojects
elementMoved
saveSplitterProportion
getArticlesByStructureId
notifyChanges
supportedReturnTypesInDefaultResolutionMode
visitParameterList
getCounterLocalService
fetchByUuid_C_Last
renderSearchTemplate
setTe
isNewArrayCreation
testDynamicQueryByPrimaryKeyMissing
getRevisionNumber
setSku
isErrorEnabled
restoreFoldingState
fetchByG_C_T_A_First
getPropertyPath
setClassNameService
helpPrinter
removeDestinationEventListeners
loadNewSSTables
hasModifierProperty
confirmOpenNewProject
getFile
compare
calcRequiredAllocations
parseReprExpression
testEmpty
te

In [17]:
max_len = 0
max_len_y = 0
word2index = {}
pcv_index_datasets_code_tmp = []
index_datasets_level_temp = []
for code, mn in zip(pcv_datasets["code"], pcv_datasets["method_name"]):
    index_code = code2index(code, pcv_vocab_dict)
#     print(mn)
    index_level = method2index(mn)
    pcv_index_datasets_code_tmp.append(index_code)
    index_datasets_level_temp.append(index_level)
    if max_len < len(index_code):
        max_len = len(index_code)
    if max_len_y < len(index_level):
        max_len_y = len(index_level)

['reset', 'Portlet', 'Invocation', 'Whitelist']
['get', 'Jdk', 'Type']
['run']
['get', 'Children']
['is', 'First']
['get', 'Class', 'Name']
['type']
['save', 'Extensions']
['get', 'Composite']
['jvm', 'Stats']
['from']
['get', 'D', 'D', 'M', 'Form', 'Field', 'Type', 'Settings']
['handle', 'Message']
['get', 'Advice']
['executor']
['set', 'Category', 'Id']
['print']
['create', 'D', 'D', 'M', 'Form']
['set', 'Review', 'Date']
['configure', 'Pointcut', 'Parameters']
['set', 'Name']
['get', 'Id']
['get', 'Buffer', 'End']
['init', 'Component']
['on', 'Complete']
['is', 'Option', 'Enabled']
['clone']
['get', 'Main', 'Xml', 'Mapping']
['add', 'Company', 'Scope', 'Permissions']
['compute']
['run']
['get', 'Contact', 'Points']
['get', 'Password', 'Encryptor']
['test', 'Get', 'Descendants']
['test', 'Interf']
['set', 'Read', 'Only']
['test', 'Most', 'Specific', 'Varargs', 'Case']
['visit', 'Single', 'Grouping', 'Set']
['get', 'Next', 'Sibling', 'In', 'Tag']
['test', 'Public', 'Schedule', 'Method

In [19]:
pcv_index_datasets_code = []
index_datasets_level = []
for code in tqdm(pcv_index_datasets_code_tmp):
    for i in range(max_len - len(code)):
#         code.insert(0, 0) # 前パディング
       code.append(0) # 後ろパディング
    pcv_index_datasets_code.append(code)
for level in tqdm(index_datasets_level_temp):
    for i in range(max_len_y - len(level)):
#         level.insert(0, 0) # 前パディング
       level.append(0) # 後ろパディング
    index_datasets_level.append(level)

100%|██████████| 10000/10000 [00:00<00:00, 840120.98it/s]


In [114]:
len(pcv_train_x[0])

2265

In [20]:
pcv_train_x, pcv_test_x, pcv_train_y, pcv_test_y = train_test_split(pcv_index_datasets_code, index_datasets_level, train_size=0.7)

In [41]:
with open(path + 'pcv/train_x.pickle', 'wb') as file:
     pickle.dump(pcv_train_x, file)
with open(path + 'pcv/train_y.pickle', 'wb') as file:
     pickle.dump(pcv_train_y, file)
with open(path + 'pcv/test_x.pickle', 'wb') as file:
     pickle.dump(pcv_test_x, file)
with open(path + 'pcv/test_y.pickle', 'wb') as file:
     pickle.dump(pcv_test_y, file)
with open(path + 'pcv/word2index.dict', 'wb') as file:
     pickle.dump(word2index, file)

In [12]:
# # GPU使えるように。
device = torch.device("cuda:0")
encoder = Encoder(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, pcv_weight)
decoder = Decoder(DEC_VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, pcv_weight)
encoder = encoder.to(device)
decoder = decoder.to(device)

# 損失関数
criterion = nn.CrossEntropyLoss()

# 最適化
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)

In [13]:
BATCH_NUM = 50
EPOCH_NUM = 300

In [48]:
all_losses = []
print("training ...")
for epoch in range(1, EPOCH_NUM+1):
    epoch_loss = 0 # epoch毎のloss

    # データをミニバッチに分ける
    input_batch, output_batch = train2batch(pcv_train_x, pcv_train_y, batch_size=BATCH_NUM)

    for i in range(len(input_batch)):
        print(f"i : {i} / {len(input_batch)}")
        # 勾配の初期化
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # データをテンソルに変換
        input_tensor = torch.tensor(input_batch[i], device=device)
        output_tensor = torch.tensor(output_batch[i], device=device)
#         input_tensor = torch.tensor(input_batch[i])
#         output_tensor = torch.tensor(output_batch[i])

        # Encoderの順伝搬
        encoder_state = encoder(input_tensor)
#         print(len(encoder_state[1][0]))

        # Decoderで使うデータはoutput_tensorを１つずらしたものを使う
        # Decoderのインプットとするデータ
        source = output_tensor[:, :-1]

        # Decoderの教師データ
        # 生成開始を表す"_"を削っている
        target = output_tensor[:, 1:]

        loss = 0
        # 学習時はDecoderはこのように１回呼び出すだけでグルっと系列をループしているからこれでOK
        # sourceが４文字なので、以下でLSTMが4回再帰的な処理してる
        decoder_output, _ = decoder(source, encoder_state)
        # decoder_output.size() = (100,4,13)
        # 「13」は生成すべき対象の文字が13文字あるから。decoder_outputの3要素目は
        # [-14.6240,  -3.7612, -11.0775,  ...,  -5.7391, -15.2419,  -8.6547]
        # こんな感じの値が入っており、これの最大値に対応するインデックスを予測文字とみなす

        for j in range(decoder_output.size()[1]):
#             print(f"J:{j}")
            # バッチ毎にまとめてloss計算
            # 生成する文字は4文字なので、4回ループ
            loss += criterion(decoder_output[:, j, :], target[:, j])

        epoch_loss += loss.item()

        # 誤差逆伝播
        loss.backward()

        # パラメータ更新
        # Encoder、Decoder両方学習
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        break
    break

    # 損失を表示
    with open('method_progres.data', 'a') as file:
        dt_now = datetime.datetime.now()
        file.write(dt_now.strftime('%Y年%m月%d日 %H:%M:%S') + "\n")
        file.write(f"epoch:{epoch}\tloss:{all_losses}\n")
    print("Epoch %d: %.2f" % (epoch, epoch_loss))
    all_losses.append(epoch_loss)
    if epoch_loss < 1: break
print("Done")

training ...
i : 0 / 140
50
Done


In [47]:
torch.save(encoder.state_dict(), path + 'pcv/lstm_pcv_encoder.model')
torch.save(decoder.state_dict(), path + 'pcv/lstm_pcv_decoder.model')
with open(path + 'pcv/loss_pcv_model.loss', 'wb') as file:
    pickle.dump(all_losses, file)

In [15]:
len(pcv_train_x)

7000

In [50]:
# Decoderのアウトプットのテンソルから要素が最大のインデックスを返す。つまり生成文字を意味する
def get_max_index(decoder_output):
  results = []
  for h in decoder_output:
    results.append(torch.argmax(h))
  return torch.tensor(results, device=device).view(BATCH_NUM, 1)

In [52]:
# 評価用データ
test_input_batch, test_output_batch = train2batch(pcv_test_x, pcv_test_y, batch_size=BATCH_NUM)
input_tensor = torch.tensor(test_input_batch, device=device)

predicts = []
for i in range(len(test_input_batch)):
  with torch.no_grad(): # 勾配計算させない
    input_tensor = torch.tensor(test_input_batch[i], device=device)
    encoder_state = encoder(input_tensor)

    # Decoderにはまず文字列生成開始を表す"_"をインプットにするので、"_"のtensorをバッチサイズ分作成
    start_char_batch = [[0] for _ in range(BATCH_NUM)]
    decoder_input_tensor = torch.tensor(start_char_batch, device=device)

    # 変数名変換
    decoder_hidden = encoder_state
    
#     print(len(decoder_hidden[1][0]))
#     print(len(decoder_input_tensor))

    # バッチ毎の結果を結合するための入れ物を定義
    batch_tmp = torch.zeros(50,1, dtype=torch.long, device=device)
    # print(batch_tmp.size())
    # (100,1)

    for _ in range(5):
      decoder_output, decoder_hidden = decoder(decoder_input_tensor, decoder_hidden)
      # 予測文字を取得しつつ、そのまま次のdecoderのインプットとなる
      decoder_input_tensor = get_max_index(decoder_output.squeeze())
      # バッチ毎の結果を予測順に結合
      batch_tmp = torch.cat([batch_tmp, decoder_input_tensor], dim=1)

    # 最初のbatch_tmpの0要素が先頭に残ってしまっているのでスライスして削除
    predicts.append(batch_tmp[:,1:])

# バッチ毎の予測結果がまとまって格納されてます。
print(len(predicts))
# 150
print(predicts[0].size())
# (100, 5)

60
torch.Size([50, 5])


In [63]:
id2char = word2index
row = []
for i in range(len(test_input_batch)):
  batch_input = test_input_batch[i]
  batch_output = test_output_batch[i]
  batch_predict = predicts[i]
  for inp, output, predict in zip(batch_input, batch_output, batch_predict):
#     print(inp)
#     x = [pcv_vocab_dict[str(idx)] for idx in inp]
#     y = [id2char[str(idx)] for idx in output]
#     p = [id2char[str(idx.item())] for idx in predict]
    pr = [idx.item() for idx in predict]
    print(output)
    print(pr)
    x_str = inp
    y_str = output
    p_str = pr

    judge = "O" if y_str == p_str else "X"
    row.append([x_str, y_str, p_str, judge])
predict_df = pd.DataFrame(row, columns=["input", "answer", "predict", "judge"])

# 正解率を表示
print(len(predict_df.query('judge == "O"')) / len(predict_df))
# 0.8492
# 間違えたデータを一部見てみる
print(predict_df.query('judge == "X"').head(10))

[4, 516, 102, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[4, 1243, 98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[4, 40, 1890, 242, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[29, 280, 84, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1240, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[70, 632, 316, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[4, 1356, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[2663, 1380, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[2005, 1007, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[4, 682, 85, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[388, 203, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[4, 849, 460, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[4, 371, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[4, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[4, 1600, 252, 0, 0, 0, 0, 0, 0, 0,

In [55]:
pcv_vocab_dict

{'RETURN,BlockStatement': 0,
 'MethodCall,RETURN': 1,
 ',delegate': 2,
 'MethodCall,BlockStatement': 3,
 'getService,MethodCall': 4,
 ',getService': 5,
 ',userId': 6,
 ',event': 7,
 'TRY,BlockStatement': 8,
 'BlockStatement,TRY': 9,
 'THROW,BlockStatement': 10,
 'VarDecl,VarDeclExpr': 11,
 ',StringBundler': 12,
 ',sb': 13,
 '3,StringBundler(3)': 14,
 ',3': 15,
 ',append': 16,
 'toString,MethodCall': 17,
 ',toString': 18,
 'VarDeclExpr,BlockStatement': 19,
 ',final': 20,
 '&&,RETURN': 21,
 'MethodCall,&&': 22,
 ',contains': 23,
 'this,MethodCall': 24,
 '=,BlockStatement': 25,
 ',this': 26,
 ',text': 27,
 'null,VarDecl': 28,
 ',null': 29,
 'SWITCH,BlockStatement': 30,
 ',int': 31,
 ',2': 32,
 'MethodCall,VarDecl': 33,
 ',input': 34,
 '1,MethodCall': 35,
 ',1': 36,
 'IF,IF': 37,
 '1,=': 38,
 'BlockStatement,IF': 39,
 '0,>': 40,
 ',0': 41,
 'true,=': 42,
 ',true': 43,
 'text,RETURN': 44,
 'SwitchCase,SWITCH': 45,
 'BREAK,SwitchCase': 46,
 'IF,BlockStatement': 47,
 ',match': 48,
 'RETURN,IF